In [1]:
'''poker engine'''

# pylint: disable=E1101, E1601, W0612

import gc
import mysql.connector
from pypokerengine.api.emulator import Emulator

import funcs_db

In [2]:
gc.collect()

22

# Open connection to poker DB

In [3]:
poker_db = mysql.connector.connect(user='root', host='127.0.0.1', database='poker_version3')
sql_path = 'c:\\Users\\adam.sohonyai\\Documents\\GitHub\\poker_model\\sql\\version3\\'

## Delete data from database tables 

In [6]:
tables = funcs_db.sql_select_tables(poker_db=poker_db)
for table in tables:
    funcs_db.sql_delete_all(poker_db=poker_db, table=table)

# Initialization

In [8]:
emulator = Emulator()

## Create game and table

In [ ]:
player_num = 6
max_round = 1
small_blind_amount = 10
ante_amount = 0

stack = 2000
players_info = {
    'uuid-0': {'name': 'player0', 'stack': stack},
    'uuid-1': {'name': 'player1', 'stack': stack},
    'uuid-2': {'name': 'player2', 'stack': stack},
    'uuid-3': {'name': 'player3', 'stack': stack},
    'uuid-4': {'name': 'player4', 'stack': stack},
    'uuid-5': {'name': 'player5', 'stack': stack}
}

summary = list()

index = funcs_db.sql_games_max_id(poker_db=poker_db)

In [ ]:
emulator.set_game_rule(player_num=player_num, max_round=max_round, small_blind_amount=small_blind_amount, \
                       ante_amount=ante_amount)
initial_state = emulator.generate_initial_game_state(players_info)
for i, player in enumerate(initial_state['table'].seats.players):

    funcs_db.sql_insert_games(poker_db=poker_db, index=index + 1, player_num=player_num, small_blind_amount=small_blind_amount, \
                              ante_amount=ante_amount, uuid=player.uuid, name=player.name, stack=player.stack, position=i, \
                              card1='', card2='', hand_db_format='', flop1='', flop2='', flop3='', turn='', river='', \
                              final_stack=0)

In [ ]:
game_state, events = emulator.start_new_round(initial_state)
for i, player in enumerate(game_state['table'].seats.players):
    
    card_list = [player.hole_card[0].RANK_MAP[player.hole_card[0].rank] + \
                 player.hole_card[0].SUIT_MAP[player.hole_card[0].suit], \
                 player.hole_card[1].RANK_MAP[player.hole_card[1].rank] + \
                 player.hole_card[1].SUIT_MAP[player.hole_card[1].suit]
    ]
    card_list.sort()
    card1 = card_list[0]
    card2 = card_list[1]
    if card1[-1] == card2[-1]:
        hand_db_format = card1[:-1] + card2[:-1] + 's'
    else:
        hand_db_format = card1[:-1] + card2[:-1] + 'o'

    funcs_db.sql_update_games_cards(poker_db=poker_db, index=index + 1, uuid=player.uuid, \
                                    card1=card1, card2=card2, hand_db_format=hand_db_format)

## Small blind and big blind moves

In [ ]:
nr = 0
step = 0
pot = 0
for i, player in enumerate(game_state['table'].seats.players):
    
    try:
        if player.serialize()[4][0]['action'] in ['SMALLBLIND', 'BIGBLIND']:
            if player.serialize()[4][0]['action'] == 'SMALLBLIND':
                amount = small_blind_amount
            else:
                amount = 2 * small_blind_amount
            funcs_db.sql_insert_history(poker_db=poker_db, phase='preflop', nr=nr, step=step, uuid=player.serialize()[1], \
                                        position=i, stack=2000, pot=pot, flop1='', flop2='', flop3='', turn='', river='', \
                                        action=player.serialize()[4][0]['action'], amount=amount, \
                                        new_stack=2000 - amount, new_pot=pot + amount)

            nr = nr + 1
            step = step + 1
            pot = pot + amount
    except:
        pass

# Game phase

In [ ]:
# poker_version2.possible_moves.bet_amount: implies the "until size" of some action, e.g. "raise til 100 
#     (already debt amount + raise amount) - emeles 100-ig (fizetendo + tovabbi novekmeny)"
# poker_version2.history.amount: net change in pot caused by some action

while events[-1]['type'] != 'event_game_finish':
    
    # data source
    data = events[-1]
    
    # calculate parameters from current game state
    phase = data['round_state']['street']
    position = data['round_state']['next_player']
    uuid = data['uuid']
    for item in data['round_state']['seats']:

        if item['uuid'] == data['uuid']:
            stack = item['stack']

    side = 0
    if data['round_state']['pot']['side'] != []:
        for amount in data['round_state']['pot']['side']:
        
            side = side + amount['amount']

    pot = data['round_state']['pot']['main']['amount'] + side
    valid_actions = funcs_db.valid_actions_check(actions=data['valid_actions'], phase=phase, position=position, \
                                                 stack=stack, small_blind_amount=small_blind_amount)
    community_cards = funcs_db.community_cards_eval(board=data['round_state']['community_card'])

    # calculate action and bet_amount and apply them on game state
    action, bet_amount, decision = funcs_db.decision_point_based_action(poker_db=poker_db, phase=phase, nr=nr, step=step, \
                                                                        position=position, stack=stack, pot=pot, \
                                                                        flop1=community_cards[0], flop2=community_cards[1], \
                                                                        flop3=community_cards[2], turn=community_cards[3], \
                                                                        river=community_cards[4], \
                                                                        valid_actions=valid_actions)
    game_state, events = emulator.apply_action(game_state=game_state, action=action, bet_amount=bet_amount)
    summary.append(decision)
    
    # calculate new parameters from current game state
    if events[-1]['type'] == 'event_game_finish':
        data = events[0]
    else:
        data = events[-1]
    
    new_phase = data['round_state']['street']    
    new_side = 0
    if data['round_state']['pot']['side'] != []:
        for amount in data['round_state']['pot']['side']:
        
            new_side = new_side + amount['amount']

    new_pot = data['round_state']['pot']['main']['amount'] + new_side
    for item in data['round_state']['seats']:

        if item['uuid'] == uuid and data['round_state']['street'] != 'showdown':
            new_stack = item['stack']
        elif item['uuid'] == uuid and data['round_state']['street'] == 'showdown':
            new_stack = stack - new_pot + pot
        else:
            pass
    
    amount = stack - new_stack
    
    # insert action in history table with calculated old and new parameters
    funcs_db.sql_insert_history(poker_db=poker_db, phase=phase, nr=nr, step=step, uuid=uuid, position=position, \
                                stack=stack, pot=pot, flop1=community_cards[0], flop2=community_cards[1], \
                                flop3=community_cards[2], turn=community_cards[3], river=community_cards[4], \
                                action=action.upper(), amount=amount, new_stack=new_stack, new_pot=new_pot)

    # increase nr parameter - nr has to be set to 0 when phase changes
    if phase == new_phase:
        nr = nr + 1
    else:
        nr = 0
    
    step = step + 1

In [ ]:
funcs_db.summarize(poker_db=poker_db, final_stacks=events[-1]['players'], summary=summary)

In [ ]:
# when player's stack is zero then rebuy, mulitply small blind with random integer in range min-max starting stack
# prepare script to run it for n loop - finsih stack is starting stack in new round
# compress stack, pot, raise/call
    # stack: 0-25, 25-50, 50-75, 75-100, 100-125, 125-150, 150-175, 175-200, 200+ ranges x small_blind
    # pot: 0-5, 5-10, 10-15, 15-20, 20-25, 25-30, 30-35, 35-40, 40-45, 45-50, 50-60, 60-70, 70-80, 80-90, 90-100, 100-120,
    #      120-140, 140-160, 160-180, 180-200, 200+ ranges x small blind
    # call: 1
    # raise: 0-0.2, 0.2-0.4, 0.4-0.6, 0.6-0.8, 0.8-1, 1-1.2, 1.2-1.4, 1.4-1.6, 1.6-1.8, 1.8-2, 2-2.5, 2.5-3, 3+
# check poker framework/module for possibiliy to compress hand/board cards

In [1]:
import sys
sys.path.append('c:\\ProgramData\\Anaconda3\\Lib\\site-packages\\deuces\\')
import deuces

In [2]:
evaluator = deuces.Evaluator()

In [3]:
# pair: Ah-As
board = [
    deuces.Card.new('Ah'),
    deuces.Card.new('Kh'),
    deuces.Card.new('2d')
]
hand = [
    deuces.Card.new('As'),
    deuces.Card.new('Qh'),
]
evaluator.evaluate(board, hand)

3335

In [6]:
# pair: Ad-As
board = [
    deuces.Card.new('Ad'),
    deuces.Card.new('Ks'),
    deuces.Card.new('2d')
]
hand = [
    deuces.Card.new('As'),
    deuces.Card.new('Qh'),
]
evaluator.evaluate(board, hand)

3335

In [4]:
# pair: Ah-As
board = [
    deuces.Card.new('Ah'),
    deuces.Card.new('Kh'),
    deuces.Card.new('2d'),
    deuces.Card.new('8c'),
]
hand = [
    deuces.Card.new('As'),
    deuces.Card.new('Qh'),
]
evaluator.evaluate(board, hand)

3329

In [5]:
# pair: Ah-As with flush draw
board = [
    deuces.Card.new('Ah'),
    deuces.Card.new('Kh'),
    deuces.Card.new('2h'),
]
hand = [
    deuces.Card.new('As'),
    deuces.Card.new('Qh'),
]
evaluator.evaluate(board, hand)

3335

In [9]:
sys.path.append('c:\\Users\\adam.sohonyai\\Documents\\GitHub\\poker_model\\py\\holdem_calc\\')
import holdem_calc
import parallel_holdem_calc

In [24]:
holdem_calc.calculate(['Ah', 'Kh', '2h'], False, 1, None, ['As', 'Qh'], True)

Winning Percentages:
(As, Qh) :  1.0
Ties:  0.0 

Player1 Histogram: 
High Card :  0.0
Pair :  0.2913968547641073
Two Pair :  0.27474560592044406
Three of a Kind :  0.04995374653098982
Straight :  0.008325624421831638
Flush :  0.34875115633672527
Full House :  0.02497687326549491
Four of a Kind :  0.0009250693802035153
Straight Flush :  0.0
Royal Flush :  0.0009250693802035153



[0.0, 1.0]

In [22]:
holdem_calc.calculate(['Ah', 'Kh', '2h', 'Ad'], False, 1, None, ['As', 'Qh'], True)

Winning Percentages:
(As, Qh) :  1.0
Ties:  0.0 

Player1 Histogram: 
High Card :  0.0
Pair :  0.0
Two Pair :  0.0
Three of a Kind :  0.5869565217391305
Straight :  0.0
Flush :  0.1956521739130435
Full House :  0.1956521739130435
Four of a Kind :  0.021739130434782608
Straight Flush :  0.0
Royal Flush :  0.0



[0.0, 1.0]

In [28]:
holdem_calc.calculate(['Ah', 'Kh', '2h', 'Ad'], False, 1, None, ['', ''], True)

Invalid card given.


NameError: name 'exit' is not defined